## Code

In [1]:
%load_ext ipython_sparql_pandas

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
from random import randint
sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-ClauCy')
sparql.setReturnFormat(JSON)

In [3]:
#from the query results extract a list that contains the counts after splitting on certain attributes
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
NegAttr=[]
AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']

def extractCountAndAttr(result):
    number = []
    listOfObj = []
    listOfPred= []
    #ret = result.queryAndConvert()
    for attr in result["results"]["bindings"]:
        listOfObj.append(attr['o'])
        listOfPred.append(attr['p'])
    return(listOfObj,listOfPred)

def generateQuestion(listOfPred, listOfObj, index, subject):
    flag = True
    if ('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>') in AttrHistory:
        return generateQuestion(listOfPred, listOfObj, index+1, subject)
        
    else:
        AttrHistory.append('<' + str(listOfPred[index]['value']) +'> <'+str(listOfObj[index]['value'])+ '>')
        
    print(f'Does the thing you are looking for have the attribute: {listOfPred[index]["value"].split("/")[-1]} {listOfObj[index]["value"].split("/")[-1]}?' )
    
    try:
        int(listOfObj[index]['value'])
    except:
        flag = False
        
    if flag is True:
        datatype = listOfObj[index]["datatype"].split("#")[-1]
        answer = findAnswer(subject, '<' + str(listOfPred[index]['value']) +'>', '"'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype)
    else:
        answer = findAnswer(subject, '<' + str(listOfPred[index]['value']) +'>', '<'+str(listOfObj[index]['value'])+ '>')
    print(answer)
    
    
    if answer:
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'.') 
        else:
            PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>.')    
        
    elif not answer:
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'. }}')
        else:
            NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>. }}')   
        
    else:
        return generateQuestion(listOfPred, listOfObj, index+1)
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)

  
def updateQuery(left, PosFilters, NegFilters):
    FilterQuestion =  (f"""
            select (count(*) as ?count) ?p ?o  where 
            {{
            
            {PosFilters}
             ?s ?p ?o .
            {NegFilters}
            }} 
            
            group by ?p ?o 
            ORDER BY ABS( {left} - ?count )
            Limit 25
            
            """) 
        
    return FilterQuestion

In [4]:
def pickSubject():
    ind=randint(0,10000)
    query = f"""
    select distinct ?s
    where {{
        ?s ?p ?o .
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    }}
    limit 10000
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"][ind:ind+1]:
        return("<"+r["s"]["value"]+">")

def findAnswer(subject,p,o):
    query = f"""
        SELECT ?s ?p ?o
        WHERE {{
        ?s ?p ?o .
            FILTER(?s = {subject})
            FILTER(?p = {p})
            FILTER(?o = {o})
            }}
            """
    #print(subject)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    res = qres["results"]["bindings"]
    return (len(res))>0

In [5]:
def numberleft(PosFilters,NegFilters):
    sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-ClauCy')
    query =  f"""
            select (count(?s) as ?count) where 
            {{
            {PosFilters}            
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
          }}       
            group by ?p ?o 
            ORDER BY DESC(?count )            
            
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        left= (r["count"]["value"])
        return(int(left))

In [6]:
def popentities(PosFilters,NegFilters):
    query =  f"""
            select ?s where 
            {{
            
            {PosFilters}
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
            
            }}
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    return qres["results"]["bindings"]

In [7]:
# resets are variables
def game_reset():
    PosAttr= ['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
    NegAttr= ['']
    AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
    left = numberleft("".join(PosAttr), "".join(NegAttr))
    FilterQuestion =  f"""
        select (count(*) as ?count) ?p ?o  where 
        {{

        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
        ?s ?p ?o .
        }} 
        group by ?p ?o 
        ORDER BY ABS( %s - ?count )
        limit 20
        """ %((left/2))
    sparql.setQuery(FilterQuestion)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    listOfObj ,listOfPred = extractCountAndAttr(qres)
    return (left, qres, PosAttr, NegAttr, AttrHistory)

In [9]:
import random

random.seed(10)
wincount = 0
gamesplayed = 0
totalgames = 5

while gamesplayed < totalgames:
    # Resets all variables
    left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
    i=0
    outofguess = []

    # pick subject
    subject = pickSubject()
    print("\n" + subject)

    while i < 20:

        # list of Pred and Obj for questions
        listOfObj ,listOfPred = extractCountAndAttr(qres)

        # if only 1 popular entity is left break
        if left<=1:
            break

        # ask question and add filters to query
        print('Question: %s' %(i+1))
        PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, 0, subject)

        # number of popular entities left
        left = numberleft(PosFilters, NegFilters)

        # run query
        sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-ClauCy')
        query = updateQuery(left/2,PosFilters, NegFilters)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        qres = sparql.queryAndConvert()

        i+=1

    # Prints answer
    if i != 20:
        for attr in popentities(PosFilters, NegFilters):
            for x in [attr['s']['value']][0:10]:
                answer = (x.split("/")[-1])
        print("Your Answer is: " + answer)
        if str(answer.split("/")[-1] + ">") == str(subject.split("/")[-1]):
            wincount += 1
            gamesplayed += 1
            print("Win!")
        else:
            gamesplayed += 1
            print("Lose...")

    # ran out of questions
    else:
        print('Final Question:')
        for j in popentities(PosFilters, NegFilters):
                outofguess.append(f'{j["s"]["value"].split("/")[-1]}')
        guess = random.choice(outofguess)
        print("Is it " + guess + "?")
        if str(subject.split("/")[-1]) == str(guess + ">"):
            wincount += 1
            gamesplayed += 1
            print('Win!')
        else:
            gamesplayed += 1
            print('Lose...')
            
print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly.")


<http://yago-knowledge.org/resource/Helena_Bonham_Carter>
Question: 1
Does the thing you are looking for have the attribute: 22-rdf-syntax-ns#type Person?
True
Question: 2
Does the thing you are looking for have the attribute: knowsLanguage English_language?
True
Question: 3
Does the thing you are looking for have the attribute: hasOccupation Actor?
False
Question: 4
Does the thing you are looking for have the attribute: hasOccupation Screenwriter?
False
Question: 5
Does the thing you are looking for have the attribute: nationality United_States?
False
Question: 6
Does the thing you are looking for have the attribute: nationality United_Kingdom?
True
Question: 7
Does the thing you are looking for have the attribute: hasOccupation Guitarist?
False
Question: 8
Does the thing you are looking for have the attribute: hasOccupation Writer?
False
Question: 9
Does the thing you are looking for have the attribute: hasOccupation Stage_actor?
True
Question: 10
Does the thing you are looking for 